In [1]:
N_JOBS = 5
N_MACHINES = 4

In [2]:
import random

def generateData(seed=3146):
    PROC = [ [ 0 for m in range(N_MACHINES) ] for j in range(N_JOBS) ]
    random.seed(seed)
    for j in range(N_JOBS):
        for m in range(N_MACHINES):
            PROC[j][m] = random.randint(1,9)
    return PROC

In [3]:
PROC = generateData(3146)

In [4]:
PROC

[[2, 7, 6, 9], [3, 2, 5, 6], [1, 4, 1, 8], [2, 4, 2, 2], [9, 5, 2, 7]]

In [5]:
# print(jobSequence())

In [6]:
import pulp

In [7]:
def val(x):
    return int(pulp.value(x))

In [8]:
prob = pulp.LpProblem("JobScheduling",pulp.LpMinimize)

`J[j]` defines the sequencing of jobs 'j' and is just a permutation of the job number:

In [9]:
JJ = pulp.LpVariable.dicts("J", (range(N_JOBS), range(N_JOBS)),
                           lowBound=0, upBound=1, cat='Integer')

In [10]:
def job(n):
    for j in range(N_JOBS):
        if val(JJ[n][j])==1:
            return j

In [11]:
def jobSequence():
    acc = []
    for j in range(N_JOBS):
        for jj in range(N_JOBS):
            if pulp.value(JJ[j][jj])==1:
                acc.append(jj)
    return acc

In [12]:
def proc(job, machine):
    global PROC
    return pulp.lpSum( [ PROC[j][machine] * JJ[j][job] for j in range(N_JOBS) ] ) 

`WAIT[j][m]` describes the wait time of job `j` before machine `m`

In [13]:
Wait = pulp.LpVariable.dicts("WAIT", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

`IDLE[j][m]` describes the idle time of machine `m` before processing job `j` 

In [14]:
Idle = pulp.LpVariable.dicts("IDLE", (range(N_JOBS), range(N_MACHINES)),
                             lowBound=0, cat='Integer')

`START[j][m]` describes the start time of machine `m` processing job `j`

In [15]:
Start = pulp.LpVariable.dicts("START", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

`STOP[j][m]` describes the stop time of machine `m` after processing job `j`

In [16]:
Stop = pulp.LpVariable.dicts("STOP", (range(N_JOBS), range(N_MACHINES)),
                          lowBound=0, cat='Integer')

In [17]:
prob += Stop[N_JOBS-1][N_MACHINES-1]

In [18]:
for j in range(N_JOBS):
    prob += pulp.lpSum( [ JJ[j][jj] for jj in range(N_JOBS) ] ) == 1
    prob += pulp.lpSum( [ JJ[jj][j] for jj in range(N_JOBS) ] ) == 1

In [19]:
for m in range(N_MACHINES):
    for j in range(N_JOBS):
        prob += pulp.lpSum( [ Idle[ji][m] + proc(ji, m) for ji in range(j) ] ) + Idle[j][m] == Start[j][m]

In [20]:
for m in range(N_MACHINES):
    for j in range(N_JOBS):
        prob += pulp.lpSum( [ Wait[j][mi] + proc(j, mi) for mi in range(m) ] ) + Wait[j][m] == Start[j][m]

In [21]:
for j in range(N_JOBS):
    for m in range(N_MACHINES):
        prob += Start[j][m] + proc(j,m) == Stop[j][m]

In [22]:
solvers = pulp.listSolvers(onlyAvailable=True) 
solver = pulp.getSolver(solvers[0], msg=0)
prob.solve(solver)

1

In [23]:
def schedule():
    row = ""
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            |" \
                f" Machine: {m:1d} |"
    row += '\n'
    for j in range(N_JOBS):
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Idle: {val(Idle[j][m]):4d} |"
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Start:{val(Start[j][m]):4d} |"  
        row += '\n' 
        row += f'| Job: {job(j):2d} |'
        for m in range(N_MACHINES):
            row += f" Wait: {val(Wait[j][m]):4d} | " \
                   f"Proc: {val(proc(j,m)):4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            row += f"       {' ':4s} | " \
                   f"Stop: {val(Stop[j][m]):4d} |"  
        row += '\n' 
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    return row

The processing times per job and machine:

In [24]:
PROC

[[2, 7, 6, 9], [3, 2, 5, 6], [1, 4, 1, 8], [2, 4, 2, 2], [9, 5, 2, 7]]

The optimal job schedule:

In [25]:
print(jobSequence())

[2, 1, 0, 3, 4]


The processing time of the optimal job schedule:

In [26]:
int(pulp.value(prob.objective))

38

The optimal job schedule in detail:

In [27]:
print(schedule())

|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    1 |            | Idle:    5 |            | Idle:    6 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   1 |            | Start:   5 |            | Start:   6 |
| Job:  2 | Wait:    0 | Proc:    1 | Wait:    0 | Proc:    4 | Wait:    0 | Proc:    1 | Wait:    0 | Proc:    8 |
|         |            | Stop:    1 |            | Stop:    5 |            | Stop:    6 |            | Stop:   14 |
|---------|------------|------------|------------|------------|---------

In [28]:
import time
 
def greedy_job_scheduling(proc):
    num_jobs = len(proc)
    num_machines = len(proc[0])
    # Initialize schedule and machine completion times
    schedule = [-1] * num_jobs  # Initialize schedule with placeholder values
    machine_completion_times = [0] * num_machines
    # Greedy selection and assignment
    for job in range(num_jobs):
        best_machine = -1
        best_completion_time = float('inf')
        for machine in range(num_machines):
            if machine_completion_times[machine] < best_completion_time:
                best_machine = machine
                best_completion_time = machine_completion_times[machine]
        schedule[job] = best_machine
        machine_completion_times[best_machine] += proc[job][best_machine]
    return schedule, machine_completion_times
 
# Example usage
proc = [[2, 7, 6, 9], [3, 2, 5, 6], [1, 4, 1, 8], [2, 4, 2, 2], [9, 5, 2, 7]]
start_time = time.time()
greedy_result, machine_completion_times = greedy_job_scheduling(proc)
print("Greedy Result:", greedy_result)
greedy_runtime = time.time() - start_time
print("Runtime:", greedy_runtime, "seconds")
 
print("Processing Times (proc):", proc)
 
# Calculate makespan
job_completion_times = [0] * len(proc[0])  # Initialize job completion times for each machine
for job, machine in enumerate(greedy_result):
    job_completion_times[machine] += proc[job][machine]
 
makespan = max(job_completion_times)
print("Makespan:", makespan)
 
# Generate schedule details for printing
idle_times = []
start_times = []
job_wait_proc = []
stop_times = []
 
for m in range(len(proc[0])):
    idle_times.append([])
    start_times.append([])
    job_wait_proc.append([])
    stop_times.append([])
 
for j, machine in enumerate(greedy_result):
    idle = machine_completion_times[machine] - proc[j][machine]
    start = machine_completion_times[machine]
    wait = start - job_completion_times[machine]
    proc_time = proc[j][machine]
    stop = start + proc_time
    idle_times[machine].append(idle)
    start_times[machine].append(start)
    job_wait_proc[machine].append((j, wait, proc_time))
    stop_times[machine].append(stop)

Greedy Result: [0, 1, 2, 3, 2]
Runtime: 0.0 seconds
Processing Times (proc): [[2, 7, 6, 9], [3, 2, 5, 6], [1, 4, 1, 8], [2, 4, 2, 2], [9, 5, 2, 7]]
Makespan: 3


In [29]:
def schedule(proc, greedy_result):
    N_JOBS = len(proc)
    N_MACHINES = len(proc[0])
    
    row = ""
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            |" \
                f" Machine: {m:1d} |"
    row += '\n'
    for j in greedy_result:  # Loop over the jobs in the greedy_result sequence
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            if greedy_result.index(j) == m:
                row += f"       {' ':4s} | " \
                       f"Idle: {0:4d} |"
            else:
                row += f"       {' ':4s} | " \
                       f"Idle: {proc[j][m]:4d} |"
        row += '\n'
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            if greedy_result.index(j) == m:
                row += f"       {' ':4s} | " \
                       f"Start:{0:4d} |"
            else:
                row += f"       {' ':4s} | " \
                       f"Start:{proc[j][m]:4d} |"
        row += '\n' 
        row += f'| Job: {j:2d} |'
        for m in range(N_MACHINES):
            if greedy_result.index(j) == m:
                row += f" Wait: {0:4d} | " \
                       f"Proc: {0:4d} |"
            else:
                row += f" Wait: {proc[j][m]:4d} | " \
                       f"Proc: {proc[j][m]:4d} |"
        row += '\n'
        row += '|         |'
        for m in range(N_MACHINES):
            if greedy_result.index(j) == m:
                row += f"       {' ':4s} | " \
                       f"Stop: {0:4d} |"
            else:
                row += f"       {' ':4s} | " \
                       f"Stop: {proc[j][m]:4d} |"
        row += '\n' 
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
                f"------------|"
    row += '\n'
    return row


In [30]:
print(schedule(proc, greedy_result))



|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    7 |            | Idle:    6 |            | Idle:    9 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   0 |            | Start:   7 |            | Start:   6 |            | Start:   9 |
| Job:  0 | Wait:    0 | Proc:    0 | Wait:    7 | Proc:    7 | Wait:    6 | Proc:    6 | Wait:    9 | Proc:    9 |
|         |            | Stop:    0 |            | Stop:    7 |            | Stop:    6 |            | Stop:    9 |
|---------|------------|------------|------------|------------|---------

In [31]:
import random
import time

def aco_job_scheduling(proc, num_ants, num_iterations, evaporation_rate, alpha, beta):
    num_jobs = len(proc)
    num_machines = len(proc[0])

    # Initialize pheromone matrix and heuristic information
    pheromone = [[1.0] * num_machines for _ in range(num_jobs)]
    heuristic = [[1.0 / proc[j][m] for m in range(num_machines)] for j in range(num_jobs)]

    best_schedule = None
    best_makespan = float('inf')

    for _ in range(num_iterations):
        ant_schedules = []

        # Construct solutions with each ant
        for ant in range(num_ants):
            schedule = [-1] * num_jobs
            machine_completion_times = [0] * num_machines

            for job in range(num_jobs):
                # Compute probabilities for job assignment to machines
                probabilities = [0.0] * num_machines
                for machine in range(num_machines):
                    if machine_completion_times[machine] == 0:
                        machine_completion_times[machine] += proc[job][machine]
                        schedule[job] = machine
                        break
                    else:
                        probabilities[machine] = (pheromone[job][machine] ** alpha) * (heuristic[job][machine] ** beta)
                else:
                    # Choose machine based on probabilities
                    total_probability = sum(probabilities)
                    selected_machine = random.choices(range(num_machines), probabilities)[0]
                    machine_completion_times[selected_machine] += proc[job][selected_machine]
                    schedule[job] = selected_machine

            ant_schedules.append(schedule)

        # Update pheromone based on ant solutions
        for ant_schedule in ant_schedules:
            ant_makespan = calculate_makespan(proc, ant_schedule)
            if ant_makespan < best_makespan:
                best_makespan = ant_makespan
                best_schedule = ant_schedule

            # Update pheromone levels
            for job in range(num_jobs):
                for machine in range(num_machines):
                    if ant_schedule[job] == machine:
                        pheromone[job][machine] = (1 - evaporation_rate) * pheromone[job][machine] + evaporation_rate / ant_makespan

    return best_schedule, best_makespan

def calculate_makespan(proc, schedule):
    num_jobs = len(proc)
    num_machines = len(proc[0])
    machine_completion_times = [0] * num_machines

    for job in range(num_jobs):
        machine = schedule[job]
        machine_completion_times[machine] += proc[job][machine]

    return max(machine_completion_times)




In [32]:
def print_schedule_formatted(idle_times, start_times, job_wait_proc, stop_times):
    num_machines = len(idle_times)
    max_jobs_per_machine = max(len(lst) for lst in job_wait_proc)
    header = "|---------|"
    for _ in range(num_machines):
        header += "------------|"
    print(header)
    print("|         |", end="")
    for m in range(num_machines):
        print(f"            | Machine: {m:2d} ", end="")
    print("|")
    print(header)
    for i in range(max_jobs_per_machine):
        print("|         |", end="")
        for m in range(num_machines):
            if i < len(idle_times[m]):
                print(f"       {' ':4s} | Idle: {idle_times[m][i]:4.1f} ", end="")
            else:
                print(f"       {' ':4s} | Idle: {' ':s} ", end="")
        print("|")
        print(header)
        print("|         |", end="")
        for m in range(num_machines):
            if i < len(start_times[m]):
                print(f"       {' ':4s} | Start: {start_times[m][i]:4.1f} ", end="")
            else:
                print(f"       {' ':4s} | Start: {' ':s} ", end="")
        print("|")
        for m in range(num_machines):
            for j in range(len(job_wait_proc[m])):
                job, wait, proc_time = job_wait_proc[m][j]
                if j == i:
                    print(f"| Job: {job:2d} | Wait: {wait:4.1f} | Proc: {proc_time:4.1f} ", end="")
                else:
                    print(f"|         |       {' ':s} | {' ':s} ", end="")
            print("|")
        print("|         |", end="")
        for m in range(num_machines):
            if i < len(stop_times[m]):
                print(f"       {' ':4s} | Stop: {stop_times[m][i]:4.1f} ", end="")
            else:
                print(f"       {' ':4s} | Stop: {' ':s} ", end="")
        print("|")
        print(header)

# Example usage
proc = [[2, 7, 6, 9], [3, 2, 5, 6], [1, 4, 1, 8], [2, 4, 2, 2], [9, 5, 2, 7]]
num_ants = 10
num_iterations = 100
evaporation_rate = 0.1
alpha = 1.0
beta = 2.0

start_time = time.time()
best_schedule, best_makespan = aco_job_scheduling(proc, num_ants, num_iterations, evaporation_rate, alpha, beta)
runtime = time.time() - start_time

print("Best Schedule:", best_schedule)
print("Best Makespan:", best_makespan)
print("Runtime:", runtime, "seconds")

# Generate schedule details for printing
idle_times = [[] for _ in range(len(proc[0]))]
start_times = [[] for _ in range(len(proc[0]))]
job_wait_proc = [[] for _ in range(len(proc[0]))]
stop_times = [[] for _ in range(len(proc[0]))]

for job, machine in enumerate(best_schedule):
    if machine >= 0:  # Ensure valid machine assignment
        if machine > 0:
            idle = sum(proc[job][:machine])
        else:
            idle = 0
        
        start = idle
        if job > 0:
            # Find the latest completion time among jobs assigned to the same machine
            previous_completion_times = [sum(proc[j][machine] for j in range(job)) if best_schedule[j] == machine else float('-inf') for j in range(job)]
            wait = start - max(previous_completion_times)
        else:
            wait = 0
        
        proc_time = proc[job][machine]
        stop = start + proc_time
        
        idle_times[machine].append(idle)
        start_times[machine].append(start)
        job_wait_proc[machine].append((job, wait, proc_time))
        stop_times[machine].append(stop)

print("Schedule after Ant Colony Optimization:")
print_schedule_formatted(idle_times, start_times, job_wait_proc, stop_times)


Best Schedule: [0, 1, 2, 3, 2]
Best Makespan: 3
Runtime: 0.011006355285644531 seconds
Schedule after Ant Colony Optimization:
|---------|------------|------------|------------|------------|
|         |            | Machine:  0             | Machine:  1             | Machine:  2             | Machine:  3 |
|---------|------------|------------|------------|------------|
|         |            | Idle:  0.0             | Idle:  3.0             | Idle:  5.0             | Idle:  8.0 |
|---------|------------|------------|------------|------------|
|         |            | Start:  0.0             | Start:  3.0             | Start:  5.0             | Start:  8.0 |
| Job:  0 | Wait:  0.0 | Proc:  2.0 |
| Job:  1 | Wait:  inf | Proc:  2.0 |
| Job:  2 | Wait:  inf | Proc:  1.0 |         |         |   |
| Job:  3 | Wait:  inf | Proc:  2.0 |
|         |            | Stop:  2.0             | Stop:  5.0             | Stop:  6.0             | Stop: 10.0 |
|---------|------------|------------|---------

In [33]:
def print_schedule_formatted(proc):
    N_JOBS = len(proc)      # Number of jobs
    N_MACHINES = len(proc[0])  # Number of machines

    # Initialize the row string
    row = ""
    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
               f"------------|"
    row += '\n'
    row += '|         |'
    for m in range(N_MACHINES):
        row += f"            | Machine: {m:1d} |"
    row += '\n'

    for j in range(N_JOBS):
        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'

        row += '|         |'
        for m in range(N_MACHINES):
            idle_time = sum(proc[j][:m])  # Idle time is sum of processing times before current machine
            row += f"       {' ':4s} | Idle: {idle_time:4d} |"
        row += '\n'

        row += '|---------|'
        for m in range(N_MACHINES):
            row += f"------------|" \
                   f"------------|"
        row += '\n'

        row += '|         |'
        for m in range(N_MACHINES):
            start_time = sum(proc[j][:m+1])  # Start time is sum of all previous processing times
            row += f"       {' ':4s} | Start:{start_time:4d} |"
        row += '\n'

        row += f'| Job: {j:2d} |'
        for m in range(N_MACHINES):
            wait_time = max(0, sum(proc[j][:m]) - sum(proc[j][:m-1]) if m > 0 else 0)  # Wait time is difference in start times
            proc_time = proc[j][m]  # Processing time on current machine
            row += f" Wait: {wait_time:4d} | Proc: {proc_time:4d} |"
        row += '\n'

        row += '|         |'
        for m in range(N_MACHINES):
            stop_time = sum(proc[j][:m+1])  # Stop time is sum of all previous processing times including current machine
            row += f"       {' ':4s} | Stop: {stop_time:4d} |"
        row += '\n'

    row += '|---------|'
    for m in range(N_MACHINES):
        row += f"------------|" \
               f"------------|"
    row += '\n'

    return row

proc = [[2, 7, 6, 9], 
        [3, 2, 5, 6], 
        [1, 4, 1, 8], 
        [2, 4, 2, 2], 
        [9, 5, 2, 7]]

# Print the formatted job schedule
schedule_table = print_schedule_formatted(proc)
print(schedule_table)


|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Machine: 0 |            | Machine: 1 |            | Machine: 2 |            | Machine: 3 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Idle:    0 |            | Idle:    2 |            | Idle:    9 |            | Idle:   15 |
|---------|------------|------------|------------|------------|------------|------------|------------|------------|
|         |            | Start:   2 |            | Start:   9 |            | Start:  15 |            | Start:  24 |
| Job:  0 | Wait:    0 | Proc:    2 | Wait:    2 | Proc:    7 | Wait:    7 | Proc:    6 | Wait:    6 | Proc:    9 |
|         |            | Stop:    2 |            | Stop:    9 |            | Stop:   15 |            | Stop:   24 |
|---------|------------|------------|------------|------------|---------